In [ ]:
pip install pyspark==3.0.0

In [ ]:
pip install psycopg2-binary randomuser

In [ ]:
import psycopg2

In [ ]:
def connect_db():
    connection = psycopg2.connect(user="datascience",
                                  password="datascience",
                                  host="datascience",
                                  port="5432",
                                  database="datascience")
    return connection

In [ ]:
create_tables = '''
DROP Table IF EXISTS "ecommerce_customer";
CREATE TABLE "ecommerce_customer" (
  "id" SERIAL PRIMARY KEY NOT NULL,
  "email" varchar(255) NOT NULL,
  "email_canonical" varchar(255) NOT NULL,
  "first_name" varchar(255) DEFAULT NULL,
  "last_name" varchar(255) DEFAULT NULL,
  "gender" varchar(1) NOT NULL DEFAULT 'u',
  "created_at" timestamp NOT NULL,
  "updated_at" timestamp DEFAULT NULL,
  "phone_number" varchar(255) DEFAULT NULL,
  "subscribed_to_newsletter" varchar(1) NOT NULL
);

CREATE UNIQUE INDEX "UNIQ_7E82D5E6E7927C74" ON "ecommerce_customer" ("email");
CREATE UNIQUE INDEX "UNIQ_7E82D5E6A0D96FBF" ON "ecommerce_customer" ("email_canonical");

'''

In [ ]:
connection = connect_db()
cursor = connection.cursor()
cursor.execute(create_tables)
connection.commit()

In [ ]:
from random import randint
import random
def create_account(mycursor, user):
    sql = '''
        INSERT INTO ecommerce_customer (email, email_canonical, first_name, last_name, gender, created_at, updated_at, phone_number, subscribed_to_newsletter)
        VALUES
        (%s, %s, %s, %s, %s, now(), now(), %s, %s);
    '''
    args = (user.get_email(), user.get_email(), user.get_first_name(), user.get_last_name(), random.choice(['m', 'f']), user.get_phone(), randint(0, 1))
    
    mycursor.execute(sql, args)


In [ ]:
from randomuser import RandomUser
def random_user_exists(mycursor):
    user = RandomUser()
    create_account(mycursor, user)

In [ ]:
connection = connect_db()
cursor = connection.cursor()
for x in range(10):
    random_user_exists(cursor)
connection.commit()

In [ ]:
from pyspark.sql import SparkSession

working_directory = 'jars/*'

spark = SparkSession.\
        builder.\
        appName("pyspark-with-mongo").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        config("spark.mongodb.input.uri","mongodb://mongodb:27017/datascience.movies").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0,org.postgresql:postgresql:42.2.18").\
        config('spark.driver.extraClassPath', working_directory). \
        config('spark.jars', working_directory).\
        getOrCreate()

sc = spark.sparkContext

In [ ]:
sc.stop()

In [ ]:
customers = spark.read.format('jdbc')\
    .option("url", "jdbc:postgresql://datascience:5432/datascience")\
    .option("user", "datascience")\
    .option("password", "datascience")\
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "ecommerce_customer")\
    .load()
    
    

In [ ]:
customers.printSchema()

In [ ]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('mongodb://mongodb:27017')
db = client.ecommerce

In [ ]:
import random
def generate_order():
    return {
        "customer_id": random.randint(0,10),
        "total": random.randint(2000, 5000),
        "lines": [{"product_id": random.randint(1,200), "quantity": random.randint(1,5)} for i in range (1, random.randint(1, 10))]
    }

In [ ]:
for x in range(10):
    order = generate_order()
    db.orders.insert_one(order)

In [ ]:
orders = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://mongodb:27017/ecommerce.orders")\
    .load()

In [ ]:
orders.select('*').show()

###
* Który customer zakupił za największą kwotę
* Czy kupujący posiadający subskrypcję do newslettera kupują więcej?

In [1]:
sc.stop()

NameError: name 'sc' is not defined